# 1.第一个express程序

## 1.1环境要求
+ nodejs16以上版本
+ 安装了npm和typescript

## 1.2安装类库
```bash
# 初始化package.json
npm init --yes

# 安装express框架.
npm i express

# 安装express的type说明文件,方便ide提示.
npm i @types/express -save-dev

# 安装生成ts-config,配置编译参数.
tsc --init

# 安装ts-node,用于直接运行typescript
npm i ts-node
```


## 1.3添加代码
```typescript
//在根目录添加index.ts
import { Request, Response } from 'express';
const express = require('express');
const app = express();

app.get('/', (req: Request, res: Response) => {
    res.send("Hello, Godance!");
});

const port = process.env.NODE_ENV ?? 3000;
app.listen(port, () => console.log(`Server listen on port:${port}`));
```

**修改package.json,添加start命令**
```json
"scripts": {
    "start":"ts-node index.ts",
    "test": "echo \"Error: no test specified\" && exit 1"
}
```

## 1.4运行程序
```bash
npm start
```

访问浏览器`http://localhost:3000`

此时在浏览器可以看到输出结果
```text
Hello, Godance!
```